# Training a neural network on MNIST with Keras

This simple example demonstrates how to plug TensorFlow Datasets (TFDS) into a Keras model.


Copyright 2020 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/datasets/keras_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/datasets/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [23]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Step 1: Create your input pipeline

Start by building an efficient input pipeline using advices from:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide


### Load a dataset

Load the MNIST dataset with the following arguments:

* `shuffle_files=True`: The MNIST data is only stored in a single file, but for larger datasets with multiple files on disk, it's good practice to shuffle them when training.
* `as_supervised=True`: Returns a tuple `(img, label)` instead of a dictionary `{'image': img, 'label': label}`.

In [24]:
(ds_train, ds_test), ds_info = tfds.load(
    'emnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Build a training pipeline

Apply the following transformations:

* `tf.data.Dataset.map`: TFDS provide images of type `tf.uint8`, while the model expects `tf.float32`. Therefore, you need to normalize images.
* `tf.data.Dataset.cache` As you fit the dataset in memory, cache it before shuffling for a better performance.<br/>
__Note:__ Random transformations should be applied after caching.
* `tf.data.Dataset.shuffle`: For true randomness, set the shuffle buffer to the full dataset size.<br/>
__Note:__ For large datasets that can't fit in memory, use `buffer_size=1000` if your system allows it.
* `tf.data.Dataset.batch`: Batch elements of the dataset after shuffling to get unique batches at each epoch.
* `tf.data.Dataset.prefetch`: It is good practice to end the pipeline by prefetching [for performance](https://www.tensorflow.org/guide/data_performance#prefetching).

In [25]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Build an evaluation pipeline

Your testing pipeline is similar to the training pipeline with small differences:

 * You don't need to call `tf.data.Dataset.shuffle`.
 * Caching is done after batching because batches can be the same between epochs.

In [26]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Step 2: Create and train the model

Plug the TFDS input pipeline into a simple Keras model, compile the model, and train it.

In [27]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(62, "softmax")
  #tf.keras.layers.Dense(62)
])
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    #optimizer=tf.keras.optimizers.Adam(0.001),
    #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath='tfKerasChars10Epochs', save_best_only=True, monitor="val_loss"),
               tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10


2022-07-02 17:13:23.655318: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 490700 of 697932


  94/5453 [..............................] - ETA: 8s - loss: 2.2598 - sparse_categorical_accuracy: 0.4747    

2022-07-02 17:13:27.973596: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


5453/5453 [==============================] - ETA: 0s - loss: 0.7826 - sparse_categorical_accuracy: 0.7660INFO:tensorflow:Assets written to: tfKerasChars10Epochs/assets


...

5453/5453 [==============================] - 10s 2ms/step - loss: 0.5825 - sparse_categorical_accuracy: 0.8139 - val_loss: 0.5780 - val_sparse_categorical_accuracy: 0.8120
Epoch 3/10
5427/5453 [============================>.] - ETA: 0s - loss: 0.5591 - sparse_categorical_accuracy: 0.8209INFO:tensorflow:Assets written to: tfKerasChars10Epochs/assets


INFO:tensorflow:Assets written to: tfKerasChars10Epochs/assets


5453/5453 [==============================] - 10s 2ms/step - loss: 0.5592 - sparse_categorical_accuracy: 0.8208 - val_loss: 0.5773 - val_sparse_categorical_accuracy: 0.8166
Epoch 4/10
5447/5453 [============================>.] - ETA: 0s - loss: 0.5526 - sparse_categorical_accuracy: 0.8228INFO:tensorflow:Assets written to: tfKerasChars10Epochs/assets


INFO:tensorflow:Assets written to: tfKerasChars10Epochs/assets


5453/5453 [==============================] - 10s 2ms/step - loss: 0.5526 - sparse_categorical_accuracy: 0.8227 - val_loss: 0.5663 - val_sparse_categorical_accuracy: 0.8209
Epoch 5/10
5453/5453 [==============================] - 10s 2ms/step - loss: 0.5542 - sparse_categorical_accuracy: 0.8229 - val_loss: 0.5853 - val_sparse_categorical_accuracy: 0.8215
Epoch 6/10
5453/5453 [==============================] - 10s 2ms/step - loss: 0.5570 - sparse_categorical_accuracy: 0.8224 - val_loss: 0.5897 - val_sparse_categorical_accuracy: 0.8200
Epoch 7/10
5453/5453 [==============================] - 10s 2ms/step - loss: 0.5614 - sparse_categorical_accuracy: 0.8213 - val_loss: 0.5904 - val_sparse_categorical_accuracy: 0.8175


In [28]:
converter = tf.lite.TFLiteConverter.from_saved_model('tfKerasChars10Epochs')
tflite_model = converter.convert()

with open("tfKerasChars10Epochs.tflite", "wb") as fp:
    fp.write(tflite_model)

2022-07-02 17:14:41.340167: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-07-02 17:14:41.340203: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-07-02 17:14:41.340327: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: tfKerasChars10Epochs
2022-07-02 17:14:41.341019: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-07-02 17:14:41.341032: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: tfKerasChars10Epochs
2022-07-02 17:14:41.342885: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-02 17:14:41.364967: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: tfKerasChars10Epochs
2022-07-02 17:14:41.372741: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. 

In [31]:
import numpy as np

model = tf.keras.models.load_model('tfKerasChars10Epochs')
model.summary()

data = np.random.randint(0, 255, (1, 28, 28))/255

print(data.shape)
#print(data[1][1])

erg = model.predict(data)
#print(np.argmax(erg))
print(erg)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 128)               100480    
                                                                 
 dense_11 (Dense)            (None, 62)                7998      
                                                                 
Total params: 108,478
Trainable params: 108,478
Non-trainable params: 0
_________________________________________________________________
(1, 28, 28)


1/1 [==============================] - 0s 28ms/step
[[1.02441635e-13 0.00000000e+00 5.47484998e-18 4.87051734e-30
  0.00000000e+00 5.31437337e-01 0.00000000e+00 2.99297866e-23
  1.13329812e-21 6.92587877e-25 1.52063853e-37 7.94872957e-13
  9.13900706e-17 3.89150534e-09 0.00000000e+00 9.22914998e-32
  0.00000000e+00 0.00000000e+00 1.13935376e-36 5.67550239e-13
  7.83443526e-26 0.00000000e+00 7.25341643e-17 5.25804297e-27
  1.46203765e-30 1.11484025e-11 1.84734412e-21 2.42480787e-21
  1.93030457e-16 4.52218557e-25 1.37205755e-17 0.00000000e+00
  5.06277001e-26 4.73101634e-37 4.68562633e-01 1.19939495e-16
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.71357346e-20 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 4.41494162e-30
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0

In [30]:
import numpy as np

interpreter = tf.lite.Interpreter(model_path="tfKerasChars10Epochs.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], data.astype(np.float32))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[3.2840946e-12 0.0000000e+00 2.8307582e-16 1.4726708e-27 1.7570290e-37
  5.7395095e-01 0.0000000e+00 9.9190867e-19 1.3116886e-15 3.5412107e-21
  5.1929966e-30 4.6734830e-12 3.2575218e-10 1.1158881e-11 0.0000000e+00
  4.2661458e-19 9.6467449e-35 0.0000000e+00 8.0863271e-37 1.7731799e-17
  2.2989868e-23 0.0000000e+00 5.9021679e-17 1.4047179e-27 8.1762151e-30
  1.0311750e-01 5.8505585e-19 3.5123919e-14 1.7707598e-10 3.0649024e-17
  2.8156990e-18 0.0000000e+00 2.8156416e-21 8.9666709e-30 3.2293147e-01
  1.5895392e-12 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.4481501e-16 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 4.0526816e-37 0.0000000e+00
  1.3358991e-27 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
